In [1]:
def readJson(f):
    for l in open(f):
        yield eval(l)

In [6]:
# from collections import OrderedDict
userdict = {}
itemdict = {}
for l in readJson('amazon_reviews_Electronics/train.json'):
    user,item,rating = l['reviewerID'],l['asin'],l['overall']
    if user not in userdict:
        userdict[user] = {'bias':0}
    userdict[user][item] = rating
    if item not in itemdict:
        itemdict[item] = {'bias':0}
    itemdict[item][user] = rating

In [7]:
# compute global mean mu
sum_score = 0
count = 0
for item in itemdict:
    for user in itemdict[item]:
        count += 1
        sum_score += itemdict[item][user]
    count -= 1 # subtract 'bias'
mu = sum_score/count
mu

3.837538

In [9]:
# compute user and item bias
l = 1 #lambda
while True:
    diff = 0
    for item in itemdict:
        sum_rating = 0
        for user in itemdict[item]:
            if user != 'bias':
                sum_rating += itemdict[item][user] - mu - userdict[user]['bias']
        tmp = itemdict[item]['bias']
        itemdict[item]['bias'] = sum_rating/(l + len(itemdict[item]) - 1)
        diff += abs(tmp - itemdict[item]['bias'])
    for user in userdict:
        sum_rating = 0
        for item in userdict[user]:
            if item != 'bias':
                sum_rating += userdict[user][item] - mu - itemdict[item]['bias']
        userdict[user]['bias'] = sum_rating/(l + len(userdict[user]) - 1)
    if diff < 1e-10:
        break

In [11]:
# compute item-item similarity
from math import sqrt
item_sim = {}
sums = {}
for user in userdict:
    for ii in userdict[user]:
        if ii != 'bias':
            bui = mu + itemdict[ii]['bias'] + userdict[user]['bias']
            for ij in userdict[user]:
                if ij != 'bias' and ij != ii:
                    buj = mu + userdict[user]['bias'] + itemdict[ij]['bias']
                    if ii not in item_sim:
                        item_sim[ii] = {}
                    if ij not in item_sim[ii]:
                        item_sim[ii][ij] = 0
                    item_sim[ii][ij] += (userdict[user][ii] - bui) * (userdict[user][ij] - buj)
                    if str(ii)+' '+str(ij) not in sums:
                        sums[str(ii)+' '+str(ij)] = {ii: 0, ij: 0,'count': 0}
                    sums[str(ii)+' '+str(ij)]['count'] += 1
                    sums[str(ii)+' '+str(ij)][ii] += (userdict[user][ii] - bui)**2
                    sums[str(ii)+' '+str(ij)][ij] += (userdict[user][ij] - buj)**2

In [12]:
lamb = 1
for ii in item_sim:
    for ij in item_sim[ii]:
        item_sim[ii][ij] /= sqrt(1e-16 + sums[str(ii)+' '+str(ij)][ii] * sums[str(ii)+' '+str(ij)][ij])
        item_sim[ii][ij] = item_sim[ii][ij]/2 + 0.5
        item_sim[ii][ij] *= sums[str(ii)+' '+str(ij)]['count']/(sums[str(ii)+' '+str(ij)]['count']+lamb)

In [13]:
item_sim

{'034d60445178609dad5af6b156865f0abd54540507c187afb7e54a68b1567881': {'06525444d732616c0790d7d2f7f9f92b56418195ba1f4496737f6e2793687bf9': 0.0,
  '09589865fd190fb0e0c8836b1392138cdf740d75769e9a109710ced98e8c90ae': 2.7755575615628914e-17,
  '09cc1e88403e07958fa2f71ffae9433cb3475332e64578e81fba5da06fcbc512': 3.3306690738754696e-16,
  '0ad8a6366f014ca05adb4578b69a2b9a627b53b857df13030ca6cf13934400b9': 0.0,
  '0b5b34ddae92575426e103d1c4719d7cb00ddb1900d98fd0d28d43f744f46217': 1.3877787807814457e-16,
  '0bf4e81d901a0cb2deb1bf36f6e24b869f84213e9f7a09cccbd748e206786ba9': 1.3877787807814457e-16,
  '0f5ce8a57f3cc1456a19bdb7389cb9a9b3d54f736dfd7d6065648bae29df9e49': 0.5,
  '1485cbadd9cd6b7760c5f505bef57e3852dec8270678430a69fee7d5516da057': 2.942091015256665e-15,
  '1ced3808e36b698a1f5407332f3649358fdbc57f8a39616cc6b85e35ddede781': 2.581268532253489e-15,
  '258d7db2e9fac71ca5c326e8d5031392a82246d6a937e5fd832c560928fe4c1a': 8.604228440844963e-16,
  '267a3ac8ce042379b5edf895ab4e59946c59cf151d8de4102

In [19]:
lam = 0.5
predictions = open("amazon_reviews_Electronics/predictions_Rating_1.txt", 'w')
for l in open("amazon_reviews_Electronics/pairs_Rating.txt"):
    if l.startswith("reviewerID"):
        predictions.write(l)
        continue
    user,item = l.strip().split('-')
    if user not in userdict:
        user = 'new'
    if item not in itemdict:
        item = 'new'
    bu = userdict[user]['bias'] if user != 'new' else 0
    bi = itemdict[item]['bias'] if item != 'new' else 0
    bui = mu + bu + bi
    if user!='new' and item in item_sim:
        predict = sum_sim = 0
        for other in item_sim[item]:
            buj = mu + userdict[user]['bias'] + itemdict[other]['bias']
            if other in userdict[user]:
                predict += item_sim[item][other] * (userdict[user][other] - buj)
                sum_sim += item_sim[item][other]
        predict = predict/(lam + sum_sim) + bui
    else:
        predict = bui
    predict = 5 if predict>5 else predict
    predictions.write(l.strip('\n') + ',' + str(predict) + '\n')
predictions.close()